In [2]:
# install pyspark
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317130 sha256=7790e609bfcb92faca00e55db60b37e6a888960371dbbf6fdc32f3e43e0f5ac4
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark


In [5]:
# import all libraries
import pyspark
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession, SQLContext

In [6]:
# membuat sesi baru

spark = SparkSession.builder.appName('TUGAS 7').getOrCreate()

In [16]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row

lines = spark.read.text("/content/drive/MyDrive/KULIAH/bigdata/ratings.dat").rdd
parts = lines.map(lambda row: row.value.split("::"))
ratingsRDD = parts.map(lambda p: Row(userId=int(p[0]), movieId=int(p[1]),
                                     rating=int(p[2]), timestamp=int(p[3])))
ratings = spark.createDataFrame(ratingsRDD)
(training, test) = ratings.randomSplit([0.8, 0.2])

# Build the recommendation model using ALS on the training data
# Note we set cold start strategy to 'drop' to ensure we don't get NaN evaluation metrics
als = ALS(maxIter=5, regParam=0.01, userCol="userId", itemCol="movieId", ratingCol="rating")
model = als.fit(training)

# Evaluate the model by computing the RMSE on the test data
predictions = model.transform(test)
predictions.show()

import math
result = predictions.rdd.map(lambda row: row['prediction'] - row['rating']).map(lambda x: x*x).filter(lambda x: not math.isnan(x))
mse = result.reduce(lambda x,y: x+y)


+------+-------+------+---------+----------+
|userId|movieId|rating|timestamp|prediction|
+------+-------+------+---------+----------+
|     1|   1193|     5|978300760| 4.5257287|
|     1|   1566|     4|978824330| 3.5840342|
|     1|   2321|     3|978302205| 3.7689948|
|     1|   2355|     5|978824291| 3.6951406|
|     2|     95|     2|978300143| 2.9876156|
|     2|    110|     5|978298625| 4.0177374|
|     2|    265|     4|978299026| 3.5477824|
|     2|    318|     5|978298413|  4.372512|
|     2|    356|     5|978299686| 4.3460965|
|     2|    434|     2|978300174|  3.107467|
|     2|    442|     3|978300025|  2.996327|
|     2|    498|     3|978299418| 1.9809573|
|     2|    648|     4|978299913| 3.3774252|
|     2|    902|     2|978298905|  3.851349|
|     2|    920|     5|978298775| 4.5531044|
|     2|   1124|     5|978299418| 4.0952907|
|     2|   1193|     5|978298413|  4.023419|
|     2|   1207|     4|978298478| 4.3194823|
|     2|   1247|     5|978298652| 3.8228645|
|     2|  